In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
import os
import concurrent.futures
import matplotlib.pyplot as plt


def write_nw_file(nw_file, mol, basis="6-31G*", geometry="", calc_ESP=True): 
    # Geometry Block of a Molecule
    if not geometry:
        conf = mol.GetConformer()
        nAtom = mol.GetNumAtoms()
        for atomId in range(nAtom):
            atomType = mol.GetAtomWithIdx(atomId).GetSymbol()
            x, y, z = conf.GetAtomPosition(atomId)
            geometry += "\t{}\t{:<20}\t{:<20}\t{:<20}\n".format(atomType, str(x), str(y), str(z))
    
    try:
        label = mol.GetProp("Label")
    except Exception as e:
        label = nw_file.split('.')[0]
    
    with open(nw_file, "w+") as f:
        f.write("echo\n")
        f.write("title {}_ESP_Calculation\n".format(label))
        f.write("start {}\n\n".format(label))
        f.write("geometry\n")
        f.write(geometry)
        f.write("end\n\n")
        f.write("charge {}\n\n".format(mol.GetProp("Charge")))
        f.write("basis\n\t* library {}\nend\n\n".format(basis))
        f.write("driver\n\tmaxiter 100\n\txyz {}\nend\n\n".format(label))
        f.write("dft\n\txc b3lyp\n\tmaxiter 2000\nend\n")
        f.write("task dft optimize\n\n")
        if calc_ESP:
            f.write("esp\n\trecalculate\n\trange 0.2\n\tprobe 0.1\n\tspacing 0.025\nend\n")
            f.write("task esp")
    
    return True

```csv_dir``` ： 存储Smiles的csv文件

```calc_dir``` ： 存储NWChem运行结果的目录

```espfile```：存储ESPMIN，ESPMAX的csv文件

```nw_files``` ： 存储需要运行的```.nw```文件的列表

In [ ]:
# Molecule Prepare
csv_dir = "./Catalysts/catalysts_subs.csv"                         # csv stores smiles
calc_dir = "./Catalysts"                     # dir stores output files
espfile = "./Catalysts/ESP_catalysts.csv"    # csv stores ESP outputs
 
data = pd.read_csv(csv_dir)
smiles = data["smiles"]
labels = data["label"].apply(str)
nMol = len(smiles)
try:
    charges = data["charge"]
except Exception as e:
    charges = [0 for _ in range(nMol)]

mols = []
for i in range(nMol):
    m = Chem.MolFromSmiles(smiles[i])
    m = Chem.AddHs(m)
    AllChem.EmbedMolecule(m, maxAttempts=20, useRandomCoords=True)
    AllChem.MMFFOptimizeMolecule(m)
    m.SetProp("Label", labels[i])
    m.SetProp("Charge", str(charges[i]))
    mols.append(m)

# Write .nw File for Calculating ESP
nw_files = []
for mol in mols:
    label = mol.GetProp("Label")           
    path = os.path.join(calc_dir, label)
    if not os.path.exists(path):
        os.mkdir(path)
    
    exist = False
    for root, dirs, files in os.walk(path):
        for file in files:
            if file == "{}.grid".format(label):
                exist = True
                break
        if exist:
            break
        
    if not exist:
        nw_file = os.path.join(path, label+".nw")
        flag = write_nw_file(nw_file, mol, calc_ESP=True)
        if flag:
            nw_files.append(nw_file)
            print("Create:", nw_file)
        else:
            print("Failed to create:", nw_file)
            
print("nw files to run:")
for file in nw_files:
    print(file)

In [ ]:
# Run .nw File
root_dir = os.getcwd()

def run_nw_file(nw_file):
    global root_dir
    basename = os.path.basename(nw_file)[:-3]
    os.chdir(os.path.dirname(nw_file))
    print("Running: ", basename)
    out = os.popen("nwchem {}".format(nw_file))
    with open(os.path.join(os.path.dirname(nw_file), 
                           "{}.txt".format(basename)), 
              "w+") as f:
        f.write(out.read())
    os.chdir(root_dir)
    print("Completed:", nw_file)
    
    return True

with concurrent.futures.ProcessPoolExecutor() as executor:
    res = [executor.submit(run_nw_file, file) for file in nw_files]

**读取```.grid```文件，并存储到```espfile```中**

In [ ]:
# Read .grid file
def read_grid_file(grid_file):
    try:
        with open(grid_file, 'r') as f:
            data = f.read().split("\n")[1:-1]

        esp = np.array([float(d.split()[-1]) for d in data])

        # Hatree(a.u.) -> kcal/mol
        esp = esp * 27.2114 * 1.6022e-19 * 6.022e23 / 4.1858518 / 1000
        
        print("Read file:", grid_file)
        return esp
    except Exception as e:
        print("Fail to read:", grid_file)


gridfiles = []
for root, dirs, files in os.walk(calc_dir):
    for r, ds, fs in os.walk(root):
        for f in fs:
            if ".grid" in f and "checkpoin" not in f:
                gridfiles.append(os.path.abspath(os.path.join(r, f)))
gridfiles = list(set(gridfiles))
gridfiles.sort()

ESPMAX = {}
ESPMIN = {}
for gridfile in gridfiles:
    label = os.path.basename(gridfile)[:-5]
    esp = read_grid_file(gridfile)
    ESPMAX[label] = esp.max()
    ESPMIN[label] = esp.min()

esp_data = pd.DataFrame([ESPMAX, ESPMIN], index=["ESPMAX", "ESPMIN"])
esp_data.to_csv(espfile) 